In [1]:
import pandas as pd
import numpy as np
import math

# Listings' files

First let's define some useful lists and dictionaries

In [2]:
string_attributes = ['listing_url', 'name', 'summary',"space", "description", "notes",\
                        "transit","access","interaction","picture_url","neighbourhood_overview", "neighbourhood",\
                        "host_url", "host_name", "host_about", "host_thumbnail_url", "host_picture_url", \
                        "host_verifications", "property_type", "room_type", "bed_type",\
                        "amenities", "house_rules", "cancellation_policy", "city", "country"]
bit_attributes = ['is_business_travel_ready', 'require_guest_profile_picture', 'require_guest_phone_verification']

date_attributes = ["review_date", "host_since"]

int_attributes = ['id', 'host_id', 'accommodates', 'bathrooms', 'bedrooms', 'beds', 'square_feet', 'guests_included',\
                    'minimum_nights', 'maximum_nights']

In [3]:
data_types = {}
for a in string_attributes:
    data_types[a] = str

For now (as testing purpose) let's use the bracelona file only

In [47]:
filename = "../Dataset/barcelona_listings.csv"


file = open(filename, newline='')
df = pd.read_csv(filename, dtype=data_types)
df["city"] = "Barcelona"

In [42]:
tables_to_attributes = \
    {"Listing": {"listing_id": "id", "listing_url": "listing_url", "listing_name": "name", "listing_summary": "summary", "listing_space": "space", "listing_description": "description", "listing_notes": "notes", "listing_transit": "transit", "listing_access": "access", "listing_interaction": "interaction","listing_picture_url": "picture_url", "listing_neighbourhood_overview" : "neighbourhood_overview",\
     "accommodates": "accommodates", "bathrooms": "bathrooms", "bedrooms": "bedrooms", "beds": "beds", "square_feet": "square_feet",\
     "price": "price", "weekly_price": "weekly_price", "monthly_price": "monthly_price", "security_deposit": "security_deposit", "cleaning_fee": "cleaning_fee", "guests_included": "guests_included", "extra_people": "extra_people",\
     "rules": "house_rules", "minimum_nights": "minimum_nights", "maximum_nights": "maximum_nights", "is_business_travel_ready": "is_business_travel_ready", "require_guest_profile_picture": "require_guest_profile_picture", "require_guest_phone_verification": "require_guest_phone_verification",\
     "review_scores_rating": "review_scores_rating", "review_scores_accuracy": "review_scores_accuracy", "review_scores_cleanliness": "review_scores_cleanliness", "review_scores_checkin": "review_scores_checkin", "review_scores_communication": "review_scores_communication", "review_scores_location": "review_scores_location", "review_scores_value": "review_scores_value",\
     "latitude": "latitude", "longitude": "longitude",\
     "host_id": "host_id", "neighbourhood_id": "neighbourhood_id", "property_type_id": "property_type_id", "room_type_id": "room_type_id", "bed_type_id": "bed_type_id", "cancellation_policy_id": "cancellation_policy_id"},\
     "Host": {"host_id" : "host_id", "host_url" : "host_url", "host_name" : "host_name", "host_since" : "host_since", "host_about" : "host_about", "host_response_time" : "host_response_time", "host_response_rate" : "host_response_rate", "host_thumbnail_url" : "host_thumbnail_url", "host_picture_url" : "host_picture_url", "neighbourhood_id": "neighbourhood_id"},\
     "Neighbourhood": {"neighbourhood_id":"neighbourhood_id", "neighbourhood_name":"neighbourhood_name","city_id": "city_id"},\
     "City": {"city_id": "city_id", "city_name": "city", "country_id": "country_id"},\
     "Property_type": {"property_type_id": "property_type_id", "property_type_name": "property_type_name"}, \
     "Room_type" : {"room_type_id": "room_type_id", "room_type_name": "room_type_name"},\
     "Bed_type": {"bed_type_id": "bed_type_id", "bed_type_name": "bed_type_name"},\
     "Cancellation_policy": {"cancellation_policy_id": "cancellation_policy_id", "cancellation_policy_name": "cancellation_policy_name"},\
     "Country": {"country_id": "country_id", "country_code": "country_code", "country_name": "country_name"},\
     "Amenity": {"amenity_id": "amenity_id", "amenity_name": "amenity_name"},\
     "Host_verification": {"host_verification_id": "host_verification_id", "host_verification_description": "host_verification_description"},\
     "Listing_amenity_map": {"listing_id": "listing_id", "amenity_id": "amenity_id"},\
     "Host_verification_map" :{"host_id": "host_id", "host_verification_id": "host_verification_id"}}


df = df.rename(columns = {'neighborhood_overview': 'neighbourhood_overview'})

In [43]:
listings_atttributes = list(tables_to_attributes["Listing"].values())

listings_atttributes.remove("neighbourhood_id")
listings_atttributes.remove("property_type_id")
listings_atttributes.remove("room_type_id")
listings_atttributes.remove("bed_type_id")
listings_atttributes.remove("cancellation_policy_id")

normalized_attr = []
normalized_attr.append("neighbourhood")
normalized_attr.append("property_type")
normalized_attr.append("room_type")
normalized_attr.append("bed_type")
normalized_attr.append("cancellation_policy")

listings_atttributes += normalized_attr


listings = df[listings_atttributes]
listings = listings.drop_duplicates()

l = zip(range(len(listings.duplicated("id").tolist())), listings.duplicated("id"))
l2 = [i for i in l if i[1]]

KeyError: "None of [Index(['id', 'listing_url', 'name', 'summary', 'space', 'description', 'notes',\n       'transit', 'access', 'interaction', 'picture_url',\n       'neighbourhood_overview', 'accommodates', 'bathrooms', 'bedrooms',\n       'beds', 'square_feet', 'price', 'weekly_price', 'monthly_price',\n       'security_deposit', 'cleaning_fee', 'guests_included', 'extra_people',\n       'house_rules', 'minimum_nights', 'maximum_nights',\n       'is_business_travel_ready', 'require_guest_profile_picture',\n       'require_guest_phone_verification', 'review_scores_rating',\n       'review_scores_accuracy', 'review_scores_cleanliness',\n       'review_scores_checkin', 'review_scores_communication',\n       'review_scores_location', 'review_scores_value', 'latitude',\n       'longitude', 'host_id', 'neighbourhood', 'property_type', 'room_type',\n       'bed_type', 'cancellation_policy'],\n      dtype='object')] are in the [columns]"

In [32]:
def clean_float_and_int(a):
    a = str(a)
    a = a.replace("$", "")
    a = a.replace(",", "")
    return float(a)

In [37]:
def f(*a):
    for g in a:
        print(g)

In [38]:
f("a", "b", "c")

a
b
c


In [54]:
neighbourhoods = df[["neighbourhood", "city"]].append(df[["host_neighbourhood", "city"]].rename(columns={"host_neighbourhood": "neighbourhood"}))
neighbourhoods = neighbourhoods.drop_duplicates()

In [56]:
l = []
for id, row in neighbourhoods.iterrows():
    l.append((row["neighbourhood"], row["city"]))

In [55]:
neighbourhoods

,neighbourhood,city
0,El Camp de l'Arpa del Clot,Barcelona
1,La Sagrada Família,Barcelona
2,El Poblenou,Barcelona
3,Vila de Gràcia,Barcelona
4,El Baix Guinardó,Barcelona
6,La Nova Esquerra de l'Eixample,Barcelona
7,Camp d'en Grassot i Gràcia Nova,Barcelona
11,Dreta de l'Eixample,Barcelona
12,El Gòtic,Barcelona
13,L'Antiga Esquerra de l'Eixample,Barcelona


In [57]:
l

[("El Camp de l'Arpa del Clot", 'Barcelona'),
 ('La Sagrada Família', 'Barcelona'),
 ('El Poblenou', 'Barcelona'),
 ('Vila de Gràcia', 'Barcelona'),
 ('El Baix Guinardó', 'Barcelona'),
 ("La Nova Esquerra de l'Eixample", 'Barcelona'),
 ("Camp d'en Grassot i Gràcia Nova", 'Barcelona'),
 ("Dreta de l'Eixample", 'Barcelona'),
 ('El Gòtic', 'Barcelona'),
 ("L'Antiga Esquerra de l'Eixample", 'Barcelona'),
 ('La Barceloneta', 'Barcelona'),
 ('Sant Pere/Santa Caterina', 'Barcelona'),
 ('El Poble-sec', 'Barcelona'),
 ('el Fort Pienc', 'Barcelona'),
 ('El Born', 'Barcelona'),
 ('El Raval', 'Barcelona'),
 ('Diagonal Mar - La Mar Bella', 'Barcelona'),
 ('Vallcarca i els Penitents', 'Barcelona'),
 ('Sants-Montjuïc', 'Barcelona'),
 ('Sant Antoni', 'Barcelona'),
 ('El Clot', 'Barcelona'),
 ('Pedralbes', 'Barcelona'),
 ('Guinardó', 'Barcelona'),
 ('Sant Andreu de Palomar', 'Barcelona'),
 ('Provençals del Poblenou', 'Barcelona'),
 ('Horta', 'Barcelona'),
 ('El Besòs i el Maresme', 'Barcelona'),
 ('La 

In [65]:
df_berlin = pd.read_csv("../Dataset/berlin_listings_filtered.csv")
df_barcelona = pd.read_csv("../Dataset/barcelona_listings.csv")
df_madrid = pd.read_csv("../Dataset/barcelona_listings.csv")

In [72]:
df = df_berlin.append(df_barcelona).append(df_madrid)


In [145]:
# this tokenizer is built to read csv files output by the cleaning process,
# it follows our rules so is absolutely not general purpose

def tokenize(filename):
    file = open(filename, 'r')

    lines = file.readlines()

    # first extracts columns name:
    columns = tokenize_line(lines[0])
    print(columns, "  len = ", len(columns))

    for i in range(1, len(lines)):
        values = tokenize_line(lines[i])
        
        #
        print(len(values))



def tokenize_line(string):
    string_len = len(string)
    result = []
    in_string = False
    cur_elmt = ""
    i = 0
    while i < string_len:
        c = string[i]
        if (c == "," or c== "\n") and not in_string:
            cur_elmt = cur_elmt.replace("\n", "")
            result.append(cur_elmt)
            cur_elmt = ""
        else:
            if in_string and c == "'" :
                if string[i+1] == "'":
                    cur_elmt += "'"
                    i += 1
                else:
                    in_string = False
            elif not in_string and c == "'":
                in_string = True
            cur_elmt += c
        i += 1
    return result


In [146]:
test = """1,2,3,'salut, je m''appelle henri','',1\n"""
tokenize_line(test)

['1', '2', '3', "'salut, je m''appelle henri'", "''", '1']

In [4]:
import csv_tokenizer as tok

In [29]:
columns, values_list = tok.tokenize("insert/insert_reviewers.csv")

In [30]:
ids = []
for v in values_list:
    ids.append(v[0])

In [31]:
maxi = 0
for i in ids:
    i = int(i)
    if maxi < i:
        maxi = i

In [28]:
len(ids) - len(list(set(ids)))

0

In [32]:
maxi

224518248